In [ ]:
import numpy as np
import json
import random 
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Activation ,Dropout
from keras.optimizers import gradient_descent_v2

lemmatizer=WordNetLemmatizer()
intents=json.loads(open('inputs.json').read())
words=[]
classes =[]
documents =[]
ignore = ['?',',','.','!',':'] 
#Data pre processing
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore]
words=sorted(set(words))
classes=sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag=[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training = np.array(training)
train_x=list(training[:, 0])
train_y = list(training[:, 1])

#Neural Network model
model= Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd = gradient_descent_v2.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=250,batch_size=5,verbose=1)
model.save('chatbot.model')
model.save('chatbot.h5',hist)

In [3]:
import numpy as np
import json
import random 
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow import keras
from keras.models import load_model
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('inputs.json').read())
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbot.model')
#creating functions for the model to search for patterns and tags in inputs to create relevant responses
def clean_up(line):
    line_words = nltk.word_tokenize(line)
    line_words = [lemmatizer.lemmatize(word) for word in line_words]
    return line_words
def bag_words(line):
    line_words=clean_up(line)
    bag=[0]*len(words)
    for i in line_words:
        for j,word in enumerate(words):
            if word==i:
                bag[j]=1
    return np.array(bag)
def predict_class(line):
    bag=bag_words(line)
    res=model.predict(np.array([bag]))[0]
    errort=0.25
    results =[[i,r] for i,r in enumerate(res) if r>errort]
    results.sort(key=lambda x:x[1],reverse = True)
    return_list=[]
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list
def reponses(intents_list,intents_json):
    tag=intents_list[0]['intent']
    listintents=intents_json['intents']
    for i in listintents:
        if i['tag']==tag:
            result =random.choice(i['responses'])
            break
    return result



    


In [ ]:

import os
import playsound
from googletrans import Translator, constants
from datetime import datetime
from gtts import gTTS
#Taking user inputs and then generating output for user.
print("---------------------Ruby is now Online ------------------------")
Name=input("May I know your name?")
print("Hey {0} this is Ruby from VIT-Bhopal".format(Name))
InputLanguageChoice = input("Which language do you want to continue?\n To continue in English enter en \n To continue in Telugu enter te \n To continue in Hindi hi")
    


while True:
    
    ques = input("Query:")
    translator = Translator()
    Translation = translator.translate(ques,dest="en",src=InputLanguageChoice)
    query=Translation.text
    #query = input("{name} :".format(name=Name))
    if ques == 'Bye' or ques=="bye" or ques =='ok thanks':
      
      print("'Ruby:' Bye")
      
      
      print("For more information please visit: https://admission.vitbhopal.ac.in/")
      break
    else:  
      ints=predict_class(query)
      res=reponses(ints,intents)
      translator =Translator()
      translation = translator.translate(res, dest=InputLanguageChoice)
      print(f"'Ruby:'{translation.text}")
      tts=gTTS(text=translation.text,lang =InputLanguageChoice)
      date_string = datetime.now().strftime("%d%m%Y%H%M%S")
      file = "testel"+date_string+".mp3"
      tts.save(file)
      playsound.playsound(file)
      os.remove(file)
    
    





    
